In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

In [ ]:
"""
input->weights->hidden layer 1(activation function)->weights->hidden layer 2(activation)->weights->output layer

compare output to intended output -> cost or loss function (cross entropy)
optimization function (optimizer) -> reduce the cost (AdamOptimizer, SGD, AdaGrad,...)

backpropogation

feed forword + backprop = epoch (cycle)
"""

In [ ]:
data_train = mnist.train

In [ ]:
im = data_train.images

In [ ]:
im[0].shape

### Computation Graphs

In [ ]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 128

In [ ]:
x = tf.placeholder('float',[None, 784])
y = tf.placeholder('float')

In [ ]:
def neural_network_model(data):
    
    # Dictionaries for layers
    hidden_layer_1 = {'weights': tf.Variable(tf.random.normal([784, n_nodes_hl1])), 
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl1]))}
    
    hidden_layer_2 = {'weights': tf.Variable(tf.random.normal([n_nodes_hl1, n_nodes_hl2])), 
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl2]))}
    
    hidden_layer_3 = {'weights': tf.Variable(tf.random.normal([n_nodes_hl2, n_nodes_hl3])), 
                      'biases': tf.Variable(tf.random.normal([n_nodes_hl3]))}
    
    output_layer = {'weights': tf.Variable(tf.random.normal([n_nodes_hl3, n_classes])), 
                      'biases': tf.Variable(tf.random.normal([n_classes]))}
                    
    # layer = (input_data * weights) + bias
    hl1 = tf.add(tf.matmul(data, hidden_layer_1['weights']), hidden_layer_1['biases'])                
    hl1 = tf.nn.relu(hl1)
                    
    hl2 = tf.add(tf.matmul(hl1, hidden_layer_2['weights']), hidden_layer_2['biases'])  
    hl2 = tf.nn.relu(hl2)
                    
    hl3 = tf.add(tf.matmul(hl2, hidden_layer_3['weights']), hidden_layer_3['biases'])  
    hl3 = tf.nn.relu(hl3)                
                        
    out = tf.add(tf.matmul(hl3, output_layer['weights']), output_layer['biases'])   
                    
    return out

### Run

In [ ]:
def train_neural_network(x):
    #feed forward prediction
    predictions = neural_network_model(x)
    
    #backpropogation = minimizing(cost=crossentropy)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #epoch = feed forward + backprop  and loss at each epoch (epoch_loss)
    total_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(total_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                _,c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
            print('Epoch',epoch,'completed out of',total_epochs, 'loss:',epoch_loss)
            
        correct = tf.equal(tf.argmax(predictions,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [ ]:
train_neural_network(x)